# Pandas怎样实现DataFrame的Merge

Pandas的Merge，相当于Sql的Join，将不同的表按Key关联到一个表

## Merge的语法：  

pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=True, suffixes=('_x','_y'), copy=True, indicator=False, validate=None)

* left, right: 要merge的dataframe或者有name的series
* how： join类型，'left', 'right', 'outer', 'inner'
* on: join的key，left和right都需要有这个key
* left_on： left的df或者series的key
* right_on： right的df或者series的key
* left_index，right_index：使用index而不是普通的column做join
* suffixes：两个元素的后缀，如果有重名，自动添加后缀，默认是('_x','_y')

文档地址：https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html  

本次讲解提纲：

    1. 电影数据集的join实例
    2. 理解merge时一对一、一对多、多对多的数量对齐关系
    3. 理解left join、right join、 inner join、outer join的区别
    4. 如果出现非key的字段重名怎么办

## 1. 电影数据集的join实例

**电影评分数据集**  
是推荐系统研究的很好的数据集  
位于本代码目录：./datas/movielens-1m  

包含三个文件： 
 1. 用户对电影的评分数据 ratings.dat
 2. 用户本身的信息数据 users.dat
 3. 电影本身的数据 movies.dat
 
可以关联三个表，得到一个完整的大表  
数据集官方地址： https://grouplens.org/datasets/movielens/

In [1]:
import pandas as pd

In [2]:
df_ratings = pd.read_csv('./datas/movielens-1m/ratings.dat', 
                         sep='::',
                         engine='python', 
                         names='UserId::MovieId::Rating::TimeStamp'.split('::'))

In [3]:
df_ratings.head()

,UserId,MovieId,Rating,TimeStamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
df_users=pd.read_csv('./datas/movielens-1m/users.dat', 
                     sep='::', 
                     engine='python', 
                     names=['UserId', 'Gender','Age','Occupation','Zip-code'])

In [5]:
df_users.head()

,UserId,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [6]:
df_movies = pd.read_csv("./datas/movielens-1m/movies.dat",
                       sep='::',
                        engine='python',
                        names=['MovieId', 'Title','Genres']
                       )

In [7]:
df_movies.head()

,MovieId,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
df_ratings_users = pd.merge(df_ratings, df_users, how='inner', on='UserId')

In [10]:
df_ratings_users.head()

,UserId,MovieId,Rating,TimeStamp,Gender,Age,Occupation,Zip-code
0,1,1193,5,978300760,F,1,10,48067
1,1,661,3,978302109,F,1,10,48067
2,1,914,3,978301968,F,1,10,48067
3,1,3408,4,978300275,F,1,10,48067
4,1,2355,5,978824291,F,1,10,48067


In [11]:
df_ratings_users_movies = pd.merge(df_ratings_users, df_movies, left_on='MovieId', right_on='MovieId', how='inner')
df_ratings_users_movies.head()

,UserId,MovieId,Rating,TimeStamp,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


## 2. 理解merge时数量的对齐关系  

以下关系要正确理解：  
    
    * one-to-one: 一对一关系，关联的key都是唯一的
        - 比如（学号、姓名） merge（学号，年龄）
        - 结果条数为 1*1
    * one-to-many: 一对多关系，左边唯一key，右边不唯一key
        - 比如（学号、姓名） merge （学号，【语文成绩，数学成绩，英语成绩】
        - 结果条数为1*N
    * many-to-many： 多对多关系，左边右边都不是唯一的
        - 比如（学号，【语文成绩，数学成绩，英语成绩】）merge （学号，【篮球、足球、乒乓球】）
        - 结果条数为M*N

**2.1 one-to-one merge**

In [13]:
left=pd.DataFrame({'student_id':[11,12,13,14],
                     'student_name':['name_a','name_b','name_c','name_d']}
                 )
left

,student_id,student_name
0,11,name_a
1,12,name_b
2,13,name_c
3,14,name_d


In [14]:
right=pd.DataFrame({'student_id':[11,12,13,14],
                   'age':['21','22','23','24']
                   })
right

,student_id,age
0,11,21
1,12,22
2,13,23
3,14,24


In [15]:
pd.merge(left,right, on='student_id')

,student_id,student_name,age
0,11,name_a,21
1,12,name_b,22
2,13,name_c,23
3,14,name_d,24


**2.2 one-to-many merge**  
注意： 数据会被复制

In [16]:
left=pd.DataFrame({'student_id':[11,12,13,14],
                     'student_name':['name_a','name_b','name_c','name_d']}
                 )
left

,student_id,student_name
0,11,name_a
1,12,name_b
2,13,name_c
3,14,name_d


In [17]:
right=pd.DataFrame({'student_id':[11,11,11,12,12,13],
                   'marks':['CHN88',"MAT90",'ENG75','CHN66','MAT55','ENG29']
                   })
right

,student_id,marks
0,11,CHN88
1,11,MAT90
2,11,ENG75
3,12,CHN66
4,12,MAT55
5,13,ENG29


In [18]:
pd.merge(left,right, on='student_id')

,student_id,student_name,marks
0,11,name_a,CHN88
1,11,name_a,MAT90
2,11,name_a,ENG75
3,12,name_b,CHN66
4,12,name_b,MAT55
5,13,name_c,ENG29


**2.3 many-to-many merge**  
注意：结果数量会出现乘法

In [19]:
left=pd.DataFrame({'student_id':[11,11,12,12,12],
                  'major':['CS','MAT','STA','CS','ARTS']
                  })
left

,student_id,major
0,11,CS
1,11,MAT
2,12,STA
3,12,CS
4,12,ARTS


In [20]:
right=pd.DataFrame({'student_id':[11,11,11,12,12,13],
                   'marks':['CHN88',"MAT90",'ENG75','CHN66','MAT55','ENG29']
                   })
right

,student_id,marks
0,11,CHN88
1,11,MAT90
2,11,ENG75
3,12,CHN66
4,12,MAT55
5,13,ENG29


In [21]:
pd.merge(left,right, on='student_id')

,student_id,major,marks
0,11,CS,CHN88
1,11,CS,MAT90
2,11,CS,ENG75
3,11,MAT,CHN88
4,11,MAT,MAT90
5,11,MAT,ENG75
6,12,STA,CHN66
7,12,STA,MAT55
8,12,CS,CHN66
9,12,CS,MAT55
